<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1c4d32738f68f1746344fbdbeb125cec3c8234272d09b26655adc071be391dff
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-17 11:00:31
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.43 C
-------------------
Today PnL: 68.32 K (0.48%)
Current PnL: -16.42 L (-11.04%)
CY Booked + Current PnL: -5.39 L (-3.62%)
-------------------
Total profit:  2.39 L
Total loss:  -18.81 L
-------------------
Total Booked + Current PnL: 21.59 L (17.74%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.72%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.31 L (57.65%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-9.44,73.0,M-LC,4.36,223792.0,22312.0,4901.0,1.56,11.07,2.19,13.51,60.0,4.55,1.59,25.07,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.20,59.0,H-LC,10.97,173470.0,17293.0,8795.0,0.41,11.07,5.07,16.71,22.0,1.97,1.23,30.44,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,105.55,60.0,M-SC,4.26,88569.0,-12208.0,12302.0,-0.30,-12.11,13.89,0.09,245.0,-0.99,0.63,16.36,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,68.0,H-MC,6.31,114215.0,13711.0,16927.0,-0.45,13.64,14.82,30.49,79.0,0.81,0.81,50.47,MH,ATH,METALS
57,RELIANCE,1533.00,-12.16,59.0,H-LC,5.33,220288.0,9922.0,18857.0,0.48,4.72,8.56,13.68,37.0,0.53,1.57,22.02,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.11,42.0,H-MC,2.27,135747.0,-41.0,42787.0,-0.51,-0.03,31.52,31.48,107.0,-0.0,0.97,12.08,X40,ATH,INSURANCE
52,PGHH,17973.08,-30.55,54.0,H-MC,4.29,200985.0,165.0,68616.0,0.22,0.08,34.14,34.25,80.0,0.0,1.43,5.28,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-17.18,38.0,H-LC,6.13,298201.0,-17641.0,130314.0,0.97,-5.59,43.70,35.67,5.0,-0.14,2.12,7.50,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.19,46.0,H-LC,7.98,219023.0,-32745.0,85945.0,0.32,-13.01,39.24,21.13,27.0,-0.38,1.56,17.81,X40,ATH,PAINTS
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.85,213062.0,8562.0,32854.0,-0.24,4.19,15.42,20.25,10.0,0.26,1.52,13.29,X40N,BTT,PHARMA
42,ITC,452.00,-37.50,52.0,H-LC,2.56,201395.0,1257.0,19636.0,-0.30,0.63,9.75,10.44,4.0,0.06,1.43,6.23,X40,NTT,FMCG
40,INFY,2275.00,-17.09,54.0,H-LC,8.67,323632.0,10744.0,160942.0,0.54,3.43,49.73,54.87,3.0,0.07,2.30,10.31,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-2.02,64.0,H-LC,1.05,242820.0,-3331.0,82535.0,0.38,-1.35,33.99,32.18,16.0,-0.04,1.73,34.20,X200,ATH,IT
42,ITC,452.0,-37.50,52.0,H-LC,2.56,201395.0,1257.0,19636.0,-0.30,0.63,9.75,10.44,4.0,0.06,1.43,6.23,X40,NTT,FMCG
51,NESTLEIND,1377.0,-8.48,56.0,H-LC,4.09,281572.0,16146.0,40659.0,-0.07,6.08,14.44,21.40,11.0,0.40,2.00,13.61,XY25,NTT,FMCG
57,RELIANCE,1533.0,-12.16,59.0,H-LC,5.33,220288.0,9922.0,18857.0,0.48,4.72,8.56,13.68,37.0,0.53,1.57,22.02,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-20.12,50.0,H-LC,5.85,213062.0,8562.0,32854.0,-0.24,4.19,15.42,20.25,10.0,0.26,1.52,13.29,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UNIONBANK,163.00,-13.18,64.0,M-LC,7.11,156974.0,16134.0,28350.0,2.81,11.46,18.06,31.59,66.0,0.57,1.12,40.98,XY24,NTT,BANKS
72,TANLA,1963.11,-19.10,70.0,H-SC,10.90,216816.0,-53945.0,383894.0,2.81,-19.92,177.06,121.86,133.0,-0.14,1.54,69.61,AR,ATH,IT
70,SURYODAY,240.00,44.74,58.0,H-SC,2.74,133008.0,-46063.0,111793.0,2.51,-25.72,84.05,36.71,135.0,-0.41,0.95,31.97,XR,NTT,BANKS
36,IEX,219.00,-28.31,61.0,H-SC,17.10,204880.0,4926.0,92298.0,2.44,2.46,45.05,48.63,115.0,0.05,1.46,14.30,XR,NTT,MISC
53,QUESS,986.00,-39.94,52.0,M-SC,40.38,59568.0,-5438.0,156366.0,2.35,-8.37,262.50,232.18,198.0,-0.03,0.42,3.74,XY24,NTT,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3862.16,63.0,M-SC,6.15,123440.0,-23928.0,75743.0,-1.29,-16.24,61.36,35.16,236.0,-0.32,0.88,26.08,XY24,NTT,AUTO
37,INDIAMART,4850.92,-51.28,47.0,H-SC,12.02,136003.0,12667.0,121097.0,-1.22,10.27,89.04,108.45,119.0,0.10,0.97,34.92,AR,ATH,MISC
76,TITAGARH,1548.00,0.09,67.0,H-SC,4.14,164281.0,-27734.0,106618.0,-1.01,-14.44,64.90,41.08,158.0,-0.26,1.17,39.21,XY24,NTT,ENGINEERING
13,BATAINDIA,2096.00,-29.87,67.0,M-SC,5.77,104929.0,-23741.0,69043.0,-0.77,-18.45,65.80,35.20,219.0,-0.34,0.75,20.43,X40,NTT,FOOTWEAR
55,RAJOOENG,143.33,-34.72,42.0,H-SC,11.05,98830.0,-3670.0,44503.0,-0.69,-3.58,45.03,39.83,114.0,-0.08,0.70,3.30,AR,ATH,MISC


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,105.55,60.0,M-SC,4.26,88569.0,-12208.0,12302.0,-0.30,-12.11,13.89,0.09,245.0,-0.99,0.63,16.36,OX40N,NTT,DURABLES
79,UJJIVANSFB,60.0,96.03,55.0,H-SC,12.36,123876.0,-18603.0,38129.0,-0.20,-13.06,30.78,13.70,163.0,-0.49,0.88,46.21,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,61.0,H-SC,1.68,225135.0,-44532.0,80868.0,2.07,-16.51,35.92,13.47,138.0,-0.55,1.60,13.87,XY24,NTT,PAINTS
19,CERA,9475.0,-19.81,47.0,H-SC,2.14,146176.0,-29727.0,71743.0,0.25,-16.90,49.08,23.89,149.0,-0.41,1.04,26.24,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-20.01,52.0,H-MC,7.78,108110.0,-23725.0,65947.0,-0.30,-18.00,61.00,32.03,98.0,-0.36,0.77,21.09,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-25.15,60.0,H-LC,12.57,298779.0,-47177.0,122021.0,1.04,-13.64,40.84,21.64,1.0,-0.39,2.13,5.84,X40,ATH,IT
40,INFY,2275.00,-17.09,54.0,H-LC,8.67,323632.0,10744.0,160942.0,0.54,3.43,49.73,54.87,3.0,0.07,2.30,10.31,X40,BTT,IT
42,ITC,452.00,-37.50,52.0,H-LC,2.56,201395.0,1257.0,19636.0,-0.30,0.63,9.75,10.44,4.0,0.06,1.43,6.23,X40,NTT,FMCG
84,VBL,671.64,-17.18,38.0,H-LC,6.13,298201.0,-17641.0,130314.0,0.97,-5.59,43.70,35.67,5.0,-0.14,2.12,7.50,X40N,ATH,FMCG
1,ABB,7934.00,-38.11,65.0,H-LC,10.74,258720.0,-2899.0,122116.0,0.77,-1.11,47.20,45.57,7.0,-0.02,1.84,10.09,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,65.0,M-MC,6.96,228664.0,3702.0,163952.0,-0.11,1.65,71.70,74.53,192.0,0.02,1.63,34.43,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,11.47,59.0,H-MC,3.55,192350.0,27680.0,40297.0,-0.03,16.81,20.95,41.28,89.0,0.69,1.37,26.13,X40N,ATH,AC
30,HAVELLS,2069.16,1.05,62.0,H-MC,1.86,250380.0,2649.0,72410.0,-0.61,1.07,28.92,30.30,92.0,0.04,1.78,14.50,X40,ATH,ELECTRICAL
85,VOLTAS,1918.49,0.60,56.0,H-MC,2.56,211050.0,19308.0,76717.0,-0.54,10.07,36.35,50.08,99.0,0.25,1.50,17.55,XY25,ATH,AC
24,DABUR,735.00,-3.66,54.0,H-MC,6.55,209528.0,12500.0,77127.0,0.29,6.34,36.81,45.49,102.0,0.16,1.49,20.21,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,68.0,H-MC,6.31,114215.0,13711.0,16927.0,-0.45,13.64,14.82,30.49,79.0,0.81,0.81,50.47,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-39.53,58.0,M-SC,0.43,102232.0,10734.0,68015.0,0.10,11.73,66.53,86.07,223.0,0.16,0.73,48.46,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.28,47.0,H-SC,12.02,136003.0,12667.0,121097.0,-1.22,10.27,89.04,108.45,119.0,0.10,0.97,34.92,AR,ATH,MISC
87,WIPRO,420.00,-10.16,58.0,M-LC,5.58,158255.0,7310.0,102138.0,0.49,4.84,64.54,72.51,53.0,0.07,1.13,11.16,XR,NTT,IT
36,IEX,219.00,-28.31,61.0,H-SC,17.10,204880.0,4926.0,92298.0,2.44,2.46,45.05,48.63,115.0,0.05,1.46,14.30,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1891.16,67.0,L-SC,3.45,54336.0,-82841.0,82933.0,0.75,-60.39,152.63,0.07,267.0,-1.00,0.39,35.13,OX40N,NTT,JEWELLERY
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,42.77,86957.0,-26592.0,66644.0,0.04,-23.42,76.64,35.27,268.0,-0.40,0.62,112.83,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-2.32,53.0,H-SC,13.03,90859.0,-10104.0,104633.0,1.85,-10.01,115.16,93.63,148.0,-0.10,0.65,35.33,SR,ATH,CHEMICALS
49,MASFIN,398.61,-15.54,58.0,H-SC,12.99,95820.0,-2160.0,23763.0,-0.08,-2.20,24.80,22.05,152.0,-0.09,0.68,39.28,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,68.0,H-MC,6.31,114215.0,13711.0,16927.0,-0.45,13.64,14.82,30.49,79.0,0.81,0.81,50.47,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-41.65,55.0,L-SC,42.77,86957.0,-26592.0,66644.0,0.04,-23.42,76.64,35.27,268.0,-0.40,0.62,112.83,XR,NTT,HOTELS
8,ATULAUTO,844.0,3862.16,63.0,M-SC,6.15,123440.0,-23928.0,75743.0,-1.29,-16.24,61.36,35.16,236.0,-0.32,0.88,26.08,XY24,NTT,AUTO
61,SAMMAANCAP,326.0,-171.59,63.0,M-SC,2.45,81966.0,-20244.0,113638.0,0.35,-19.81,138.64,91.37,208.0,-0.18,0.58,33.06,XY25,NTT,FINANCE
54,RAJESHEXPO,518.0,1891.16,67.0,L-SC,3.45,54336.0,-82841.0,82933.0,0.75,-60.39,152.63,0.07,267.0,-1.00,0.39,35.13,OX40N,NTT,JEWELLERY
76,TITAGARH,1548.0,0.09,67.0,H-SC,4.14,164281.0,-27734.0,106618.0,-1.01,-14.44,64.90,41.08,158.0,-0.26,1.17,39.21,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.47
1,25,42.78
2,50,73.20


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.32
LC    34.03
MC    22.65
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.49
X40      15.01
X40N     12.77
XR       11.46
XY25      9.59
AR        8.54
OX40N     8.02
X200      1.73
MH        1.68
X5K       1.43
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.49
H-LC    25.86
H-MC    19.75
M-SC    13.30
M-LC     7.12
M-MC     2.60
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.24,-13.00,74.61
FMCG,11.54,-4.30,40.80
FINANCE,9.50,-15.15,60.24
BANKS,7.70,-10.82,61.83
MISC,7.34,-11.35,73.27
PAINTS,5.73,-14.58,31.66
ELECTRICAL,5.60,-4.43,61.98
AC,3.56,3.36,29.37
INSURANCE,3.36,-6.60,43.16


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2839594.0,22
XR,1301412.0,14
AR,1294662.0,9
X40,738922.0,10
X40N,617001.0,10
OX40N,518625.0,10
XY25,401654.0,7
SR,242168.0,2
X5K,116231.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3408231.0,29
M-SC,1374727.0,17
H-LC,1239091.0,15
H-MC,1175726.0,15
M-LC,377550.0,5
M-MC,313135.0,2
L-SC,246484.0,3
L-MC,60311.0,1
L-LC,35997.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1029556.0      6
M-SC       XY24       796676.0      7
H-SC       AR         780635.0      5
           XR         753718.0      7
H-MC       XY24       486738.0      4
H-LC       X40        459799.0      5
           AR         352726.0      2
M-MC       XY24       313135.0      2
H-SC       X40N       292408.0      4
           OX40N      248225.0      4
           SR         242168.0      2
H-LC       X40N       225306.0      4
H-MC       X40        210080.0      4
           XY25       187602.0      2
L-SC       XR         163551.0      2
M-SC       AR         161301.0      2
M-LC       XY24       154280.0      2
M-SC       OX40N      121520.0      4
M-LC       X5K        116231.0      1
M-SC       XY25       113638.0      1
           XR         112549.0      2
H-MC       XR         109145.0      1
M-LC       XR         102138.0      1
H-MC       X40N        99287.0      2
L-SC       OX40N       82933.0      1
H-LC       X200        82535.0      1
M-SC       X40         69043.0      1
H-MC       OX40N       65947.0      1
H-SC       MH          61521.0      1
L-MC       XR          60311.0      1
H-LC       XY25        59516.0      2
           XY24        59209.0      1
L-LC       XY25        35997.0      1
H-MC       MH          16927.0      1
M-LC       XY25         4901.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
